In [11]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import dash_table
import dash
import pandas as pd
import json
import csv
import plotly.graph_objs as go


In [23]:
# Build App
app = JupyterDash(__name__,suppress_callback_exceptions=True)

#function to query dbpedia endpoint
def sql(value, endpoint):
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(value)
    sparql.setReturnFormat(JSON)
    processed_results = sparql.query().convert()
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

#layout

app.layout = html.Div([
    
   html.Div(
        className="navbar",
        children=[ 
            html.H3(
                children="Interactive Data Analytics Dashboard for RDF Knowledge Graphs", 
                className="navbar--title"),
            html.Div(
                children=[
                dcc.Tabs(id='tabs', value='Dashboard', children=[
                    dcc.Tab(label='Dashboard', value='Dashboard'),
                    dcc.Tab(label='Tables', value='Tables'),
                    dcc.Tab(label='Charts', value='Charts'),
                    dcc.Tab(label='Query Samples', value='Samples'),
                    dcc.Tab(label='Tutorial', value='Tutorial'),
                ]),
                html.Div(id='tabs-content', className="tabs-content")
            ])
    ]),
    html.Div(
        className="querybox",
        children=[
            html.H4(
                className="querybox--title",
                children="SPARQL Query"
            ),
            dcc.Textarea(
                id= "query-endpoint", 
                value="https://query.wikidata.org/sparql",
                placeholder="Enter your SPARQL query endpoint.",
                className="querybox--endpoint"
            ),
            dcc.Textarea(
                id= "query-text", 
                value="",
                placeholder="Enter your SPARQL query.",
                className="querybox--textarea"
            ),
            html.Button(
                id="submit-btn", className="querybox--btn", 
                children="SUBMIT", 
                n_clicks=0
            ),
            html.Button(
                id="compare-btn", className="querybox--btn", 
                children="COMPARE", 
                n_clicks=0
            ),
            html.Button(
                id="upload-btn", className="querybox--btn", 
                children="UPLOAD", 
                n_clicks=0
            ),
    ]),      
    #hidden div 
    html.Div( 
    id='intermediate-value', 
    children=[],
    style={'display': 'none'})
            
])#layout div


#-----------------------------------------query samples Callback----------------------------------------------------------------
#make query examples table from csv file
with open('../data/Book3.csv', 'rt') as f:
    csv_reader = csv.reader(f)
    headers = []
    queries = []
    endpoints = []
    for line in csv_reader:
        headers.append(line[0])
        queries.append(line[1])
        endpoints.append(line[2])

    data = {'Query Examples':  headers}
    df = pd.DataFrame (data, columns = ['Query Examples'])
    ResultListdataframe = pd.DataFrame (data=[], columns = [])


#-----------------------------------------initial empty table ----------------------------------------------------------------
#initial empty table
start_table_df = pd.DataFrame(columns=[''])

#-----------------------------------------selected tab callback ----------------------------------------------------------------
#render page for selected tab
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value'),
)

#---------------------------------------------------render content--------------------------------------------------------------

def render_content(tab):
    if tab == 'Dashboard':
        return html.Div([
        ])
    elif tab == 'Tables':
        return html.Div([
            html.H3('Tables'),
            html.Div(
                id='MainTableDiv',
                children=[
                    dash_table.DataTable(
                        id='Resulttable',
                        data=start_table_df.to_dict('records'), 
                        columns = [{'id': c, 'name': c} for c in start_table_df.columns],
                        style_cell=dict(textAlign='left'),   
                        editable=True,
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                        column_selectable="single",
                        row_selectable="multi",
                        row_deletable=True,
                        selected_columns=[],
                        selected_rows=[],
                        page_action="native",
                        page_current= 0,
                        page_size= 10,
                        export_format="csv"
                    )
                ],
                style= {'display': 'none'}
            ),
        ])
    elif tab == 'Charts':
        return html.Div([
               html.H3('Charts'), 
               html.Div(
                   id="result-graph",
                   className="result-graph",
                   children=[
                        #dcc.Dropdown(
                        #id='chooseChart-dropdown',
                        #options=[
                        #{'label': 'scatter chart', 'value': 'scatter'},
                        #{'label': 'bar chart', 'value': 'bar'},
                        #{'label': 'line chart', 'value': 'line'},
                        #{'label': 'pie chart', 'value': 'pie'},
                        #{'label': 'polar chart', 'value': 'polar'}

                        #],
                        #value = 'bar',
                        #multi=False,
                        #style={'width': '40%'}
                        #),
     

                       
                        dcc.Graph(
                         id='graph',
                        ),
                
                     ],
                 style= {'display': 'none'}  
               )
  
        ])
    elif tab == 'Samples':
        return html.Div([
            html.H3('Query Samples'),
            html.Div(
                className="examples",
                children=[
                    dash_table.DataTable(
                        id='querySamples',
                        columns=[{"name": i, "id": i} for i in df.columns],
                        data=df.to_dict('records'),
                        style_table={'width': '800px'},
                        style_cell=dict(textAlign='left'), 
                        column_selectable='single'
                    )
                ])
        ])
    elif tab == 'Tutorial':
        return html.Div([
            html.H3('Tutorial'),
            html.Div(
                className="tutorial",
                children=[
                ]),
        ])
    
    
    


#---------------------------------------------------Query Samples Callback--------------------------------------------------------------
# write selected query sample in the query text area    
@app.callback(
    [Output("query-text", "value"), Output("query-endpoint", "value")],
    [Input("querySamples", "active_cell")],
    [State("query-text", "value"),State("query-endpoint", "value")]
)
def get_active_cell(active_cell,prev_query,prev_endpoint):
    if(active_cell):
        return queries[next(iter(active_cell.values()))], endpoints[next(iter(active_cell.values()))] 
    return prev_query,prev_endpoint

#---------------------------------------------------Get-data Callback--------------------------------------------------------------
@app.callback(Output("intermediate-value", 'children'),
              [Input("submit-btn", "n_clicks")],
              [State("query-text","value"), State("query-endpoint","value")])

def get_data(n_clicks, value, endpoint):
     if n_clicks > 0:
        ResultListdataframe = sql(value, endpoint)   
        print("Get-data ",ResultListdataframe)
        
        return ResultListdataframe.to_json(date_format='iso', orient='split')


#---------------------------------------------------Table Callback--------------------------------------------------------------
#generate table from query    
@app.callback(
    [Output("Resulttable", "data"), Output('Resulttable', 'columns'), Output('MainTableDiv', 'style')],
    [Input("submit-btn", "n_clicks"),
    Input("intermediate-value", "children"),
    Input('Resulttable', "derived_virtual_selected_rows")],
)    
#def gen_table(jsonified_ResultListdataframe,derived_virtual_selected_rows):
def gen_table(n_clicks, jsonified_ResultListdataframe,derived_virtual_selected_rows):
    if n_clicks > 0:
        selectedRowsIndex=derived_virtual_selected_rows
        print("table", jsonified_ResultListdataframe)
        resultListdataframe = pd.read_json(jsonified_ResultListdataframe, orient='split')
        mycolumns = [{'name': index, 'id': index,"deletable": True, "selectable": True} for index in resultListdataframe.columns]

        return resultListdataframe.to_dict('records'),mycolumns,{'display': 'block'}
    return start_table_df.to_dict('records'), [{'id': '', 'name': ''}],{'display': 'none'}

#---------------------------------------------------Charts Callback--------------------------------------------------------------
@app.callback(
    [Output("graph", "figure"), Output('result-graph', 'style')],
    [#Input("submit-btn", "n_clicks"),
    Input("intermediate-value", "children"),
    #Input("chooseChart-dropdown","value")
    ],
)
#def gen_graph(n_clicks, jsonified_ResultListdataframe):
def gen_graph(jsonified_ResultListdataframe):
    #fig = html.Div()
    #if n_clicks > 0:
    print("chart", jsonified_ResultListdataframe)
    ResultListdataframe = pd.read_json(jsonified_ResultListdataframe, orient='split')
        
        # initialize figure
    fig = go.Figure()
        
        # add trace
    fig.add_trace(go.Bar(x=ResultListdataframe.iloc[:, 0],
                             y=ResultListdataframe.iloc[:, 1],
                             name='bar'))
        
    fig.add_trace(go.Scatter(x=ResultListdataframe.iloc[:, 0],
                             y=ResultListdataframe.iloc[:, 1],
                             mode='markers',
                             name='marker',
                             visible=False))
        
    fig.add_trace(go.Scatter(x=ResultListdataframe.iloc[:, 0],
                              y=ResultListdataframe.iloc[:, 1],
                              mode='lines',
                              name='line',
                              visible=False))

       
    fig.update_layout(
        updatemenus=[
            dict(
                active=0,
                buttons=list([
                          
                dict(label="bar",
                    method="update",
                    args=[{"visible": [True, False, False]},
                    {"title": "Bar Graph",}]),
                dict(label="Scatter",
                    method="update",
                    args=[{"visible": [False, True, False]},
                    {"title": "Scatter Graph",}]),
                dict(label="line",
                    method="update",
                    args=[{"visible": [False, False, True]},
                    {"title": "Line Graph",}]),
#                         dict(label="Scatter and Bar",
#                              method="update",
#                              args=[{"visible": [True, True, False]},
#                                    {"title": "Scatter and Bar Graphs",}]),
                ]),
            )
        ])
        

    return fig,{'display': 'block'}
 

In [24]:
# Run app and display result inline in the notebook
app.run_server(mode='external',port=8080)

Dash app running on http://127.0.0.1:8080/
Get-data                                           item                   itemLabel  \
0    http://www.wikidata.org/entity/Q16007979          Mohamed Bechir-Sow   
1    http://www.wikidata.org/entity/Q19629427          Abbès Moulessehoul   
2    http://www.wikidata.org/entity/Q19630221              Abdallah Tebib   
3    http://www.wikidata.org/entity/Q19606698      Abdelkader Benazzedine   
4    http://www.wikidata.org/entity/Q22683638             Abdelkader Cadi   
..                                        ...                         ...   
594   http://www.wikidata.org/entity/Q3163882        Jean-Baptiste Biaggi   
595   http://www.wikidata.org/entity/Q3166996           Jean-Louis Tinaud   
596    http://www.wikidata.org/entity/Q642640                  Luc Chatel   
597   http://www.wikidata.org/entity/Q2569191           Raymond Patenôtre   
598  http://www.wikidata.org/entity/Q15969684  Samuel Welles de Lavalette   

         placeLabel   

chart {"columns":["item","itemLabel","placeLabel","countryLabel"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258

Get-data      yearmonth count
0      2016-7  1892
1      2001-6  1109
2      2014-2  1821
3      2012-3  1736
4      2012-2  1764
..        ...   ...
248    2015-8  1803
249   2003-11  1245
250   2018-10  1716
251    2001-4  1130
252    2016-6  1680

[253 rows x 2 columns]
chart {"columns":["yearmonth","count"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,

Get-data                                            item       itemLabel
0       http://www.wikidata.org/entity/Q378619              CC
1       http://www.wikidata.org/entity/Q498787          Muezza
2       http://www.wikidata.org/entity/Q677525         Orangey
3       http://www.wikidata.org/entity/Q851190     Mrs. Chippy
4      http://www.wikidata.org/entity/Q1050083        Catmando
..                                         ...             ...
146  http://www.wikidata.org/entity/Q100451536         Sanjuro
147  http://www.wikidata.org/entity/Q100965010  Myōbu no otodo
148  http://www.wikidata.org/entity/Q101943970          Noutti
149  http://www.wikidata.org/entity/Q101947824        Tigrette
150  http://www.wikidata.org/entity/Q101948600          Miolek

[151 rows x 2 columns]
